In [ ]:
%tensorflow_version 1.x



TensorFlow 1.x selected.


In [ ]:
'''
!cp -r "/content/drive/MyDrive/test/get_cifar.py" "/content/"
!cp -r "/content/drive/MyDrive/test/load_cifar.py" "/content/"
!cp -r "/content/drive/MyDrive/test/cifar-10-batches-py" "/content/cifar/"
!cp -r "/content/drive/MyDrive/test/get_svhn.py" "/content/"
!cp -r "/content/drive/MyDrive/test/load_svhn.py" "/content/"
!cp -r "/content/drive/MyDrive/test/inception_score.py" "/content/"
!cp -r "/content/drive/MyDrive/test/plot.py" "/content/"
!cp -r "/content/drive/MyDrive/test/stat" "/content/"
!cp "/content/drive/MyDrive/test/fid.py" "/content/"
'''

'\n!cp -r "/content/drive/MyDrive/test/get_cifar.py" "/content/"\n!cp -r "/content/drive/MyDrive/test/load_cifar.py" "/content/"\n!cp -r "/content/drive/MyDrive/test/cifar-10-batches-py" "/content/cifar/"\n!cp -r "/content/drive/MyDrive/test/WGAN-tensorflow-master/get_svhn.py" "/content/"\n!cp -r "/content/drive/MyDrive/test/WGAN-tensorflow-master/load_svhn.py" "/content/"\n!cp -r "/content/drive/MyDrive/test/inception_score.py" "/content/"\n!cp -r "/content/drive/MyDrive/test/plot.py" "/content/"\n!cp -r "/content/drive/MyDrive/test/stat" "/content/"\n!cp "/content/drive/MyDrive/test/fid.py" "/content/"\n'

In [ ]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
from six.moves import xrange
import tensorflow.contrib.slim as slim
from __future__ import print_function, division
import tensorflow as tf
import tensorflow.contrib.layers as ly
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from functools import partial
from tensorflow.examples.tutorials.mnist import input_data

import time
import collections
import pickle
import scipy.misc

#from scipy.misc import imsave
import imageio
from PIL import Image #PIL pakage name is Pillow

from load_svhn import load_svhn
from load_cifar import load_cifar
import inception_score
from inception_score import get_inception_score as fget_inception_score
import plot

import fid
get_ipython().run_line_magic('matplotlib', 'inline')
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # 图像显示大小

print(tf.__version__)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.gfile.GFile.



Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7

1.15.2


In [ ]:
batch_size = 64
noise_dim = 128

device = '/gpu:0'

# update Citers times of critic in one iter(unless i < 25 or i % 500 == 0, i is iterstep)
Citers = 5 # For WGAN and WGAN-GP, number of critic iters per gen iter
# the upper bound and lower bound of parameters in critic
clamp_lower = -0.01
clamp_upper = 0.01
# whether to use mlp or dcgan stucture
is_mlp = False



# 'wgan-gp' for gp WGAN and 'wgan' for WGAN with clip and 'dcgan' for DCGAN
mode = 'wgan'
# whether to use SVHN or MNIST, set false and MNIST is used
data = 'svhn'

if data == 'mnist':
  plt.rcParams['image.interpolation'] = 'nearest'  # 最近邻差值: 像素为正方形
  plt.rcParams['image.cmap'] = 'gray' # 使用灰度输出而不是彩色输出
  OUTPUT_DIM = 1*28*28 
  channel = 1
  s = 28 #img size

else:
  OUTPUT_DIM = 3*32*32
  channel = 3
  s = 32 #img size



# if 'wgan-gp' is chosen the corresponding lambda must be filled
lam = 10. # Gradient penalty lambda hyperparameter

if mode =='wgan-gp':
  learning_rate_G = 1e-4
  learning_rate_C = 1e-4
elif mode == 'wgan':
  learning_rate_G = 3e-5
  learning_rate_C = 3e-5
elif mode == 'dcgan':
  learning_rate_G = 2e-4
  learning_rate_C = 2e-4

 # Number of pixels in each iamge

Dim = 64
if mode == 'wgan':
  bn = 0
else:
  bn = 1
# directory to store log, including loss and grad_norm of generator and critic
log_dir = './log_wgan'
ckpt_dir = './ckpt_wgan'
img_dir = './'+mode+'_'+data
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
# max iter step, note the one step indicates that a Citers updates of critic and one update of generator
max_iter_step = 20000  # How many generator iterations to train for 

show_every = 100

FID = True
if data == 'mnist':
  FID = False

IS = False
if data == 'cifar':
  IS =True

FID_STEP = 500 # FID evaluation every FID_STEP
# FID evaluation.
FID_EVAL_SIZE = 5000 # Number of samples for evaluation
FID_SAMPLE_BATCH_SIZE = 100  # Batch size of generating samples, lower to save GPU memory
FID_BATCH_SIZE = 200 # Batch size for final FID calculation i.e. inception propagation etc.

In [ ]:
def disp_images(images,iter):
  images = np.reshape(images, [images.shape[0], -1, channel])
  sqrtn = int(np.ceil(np.sqrt(images.shape[0])))
  sqrtimg = int(np.ceil(np.sqrt(images.shape[1])))
  figure = plt.figure(figsize=(sqrtn, sqrtn))
  gs = gridspec.GridSpec(sqrtn, sqrtn)  # 指定放置子图的网格的几何形状 行、列
  gs.update(wspace=0.05, hspace=0.05)  # 子图之间宽度空间、高度空间
  for i, img in enumerate(images): # 列举数据、数据下标
    ax = plt.subplot(gs[i])
    plt.axis('off')
    ax.set_xticklabels([])
    ax.set_aspect('equal')
    if channel == 3:
      plt.imshow(img.reshape([sqrtimg, sqrtimg, channel]))
    elif channel == 1:
      plt.imshow(img.reshape([sqrtimg, sqrtimg]))
  save_path = img_dir + '/{}-{}-samples_{}.png'.format(mode,data,iter) # 要保存为的图片
  plt.savefig(save_path)
  return

In [ ]:
#leaky relu
def lrelu(x, alpha=0.01):
  a = tf.maximum(x, alpha*x)
  return a

In [ ]:
def sample_noise(batch_size,dim):
  # 从均匀分布中输出随机值 
  random_noise = tf.random_uniform(maxval=1,minval=-1,shape=[batch_size,dim])
  
  return random_noise

In [ ]:
def generator_conv(n_samples, x=None):
  if x is None:
    x = sample_noise(n_samples, noise_dim)
  with tf.variable_scope("generator"):
    # x.shape: [batch_size, z_dim] 64 128
    #print(x.shape)
    x = ly.fully_connected(x, 4 * 4 * 8 * Dim, activation_fn=lrelu, normalizer_fn=ly.batch_norm)
    #print(x.shape)
    x = tf.reshape(x, (-1, 4, 4, 8*Dim))
    #print(x.shape)
    if bn:
      x = ly.conv2d_transpose(x, 4*Dim, 3, stride=2,activation_fn=tf.nn.relu, normalizer_fn=ly.batch_norm, padding='SAME', weights_initializer=tf.random_normal_initializer(0, 0.02))
    else:
      x = ly.conv2d_transpose(x, 4*Dim, 3, stride=2,activation_fn=tf.nn.relu, padding='SAME', weights_initializer=tf.random_normal_initializer(0, 0.02))
    #print(x.shape)
    if bn:
      x = ly.conv2d_transpose(x, 2*Dim, 3, stride=2,activation_fn=tf.nn.relu, normalizer_fn=ly.batch_norm, padding='SAME', weights_initializer=tf.random_normal_initializer(0, 0.02))
    else:
      x = ly.conv2d_transpose(x, 2*Dim, 3, stride=2,activation_fn=tf.nn.relu, padding='SAME', weights_initializer=tf.random_normal_initializer(0, 0.02))
    #print(x.shape)
    if bn :
      x = ly.conv2d_transpose(x, Dim, 3, stride=2,activation_fn=tf.nn.relu, normalizer_fn=ly.batch_norm, padding='SAME', weights_initializer=tf.random_normal_initializer(0, 0.02))
    else:
      x = ly.conv2d_transpose(x, Dim, 3, stride=2,activation_fn=tf.nn.relu, padding='SAME', weights_initializer=tf.random_normal_initializer(0, 0.02))
    #print(x.shape)
    x = ly.conv2d_transpose(x, channel, 3, stride=1,activation_fn=tf.nn.tanh, padding='SAME', weights_initializer=tf.random_normal_initializer(0, 0.02))

   # print(x.name)
    #print(x.shape)
    return x

In [ ]:
def generator_mlp(n_samples,x=None):
  #hidden layer size if mlp is chosen
  if x is None:
    x = sample_noise(n_samples, noise_dim)
  with tf.variable_scope("generator"):
    train = ly.fully_connected(x, 4 * 4 * 8 * Dim, activation_fn=lrelu, normalizer_fn=ly.batch_norm)
    train = ly.fully_connected(train, 4*Dim, activation_fn=lrelu, normalizer_fn=ly.batch_norm)
    train = ly.fully_connected(train, 2*Dim, activation_fn=lrelu, normalizer_fn=ly.batch_norm)
    train = ly.fully_connected(train, s*s*channel, activation_fn=tf.nn.tanh, normalizer_fn=ly.batch_norm)
    train = tf.reshape(train, tf.stack([batch_size, s, s, channel]))
    return train

In [ ]:
def discriminator(x,  reuse=False):

  with tf.variable_scope('critic') as scope:
    if reuse:
      scope.reuse_variables()
    init = tf.contrib.layers.xavier_initializer() # 初始化器
    print('x0.shape:',x.shape)
    x = tf.reshape(x, [-1,32,32,1])
    print('x1.shape:',x.shape)
    x = tf.layers.conv2d(x, 64, 4, activation=lrelu, strides=2, padding='valid',
                         kernel_initializer=init, name='conv_0')
    print('x2.shape:',x.shape)
    x = tf.layers.conv2d(x, 128, 4, activation=lrelu, strides=2, padding='valid',
                         kernel_initializer=init, name='conv_1')
    print('x3.shape:',x.shape)
    x = tf.layers.batch_normalization(x, name='batchnorm_0')
    print('x4.shape:',x.shape)
    x = tf.reshape(x, [-1,3200])
    print('x5.shape:',x.shape)
    x = tf.layers.dense(x, 1024, activation=lrelu, kernel_initializer=init, name='dense_0')
    print('x6.shape:',x.shape)
    logits = tf.layers.dense(x, 1, kernel_initializer=init, name='logits')

    return logits

In [ ]:
def critic_conv(img, reuse=False):
  with tf.variable_scope('critic') as scope:
    if reuse:
      scope.reuse_variables()
    
    img = ly.conv2d(img, num_outputs=Dim, kernel_size=3,stride=2, activation_fn=lrelu)
    if bn :
      img = ly.conv2d(img, num_outputs=2*Dim, kernel_size=3,stride=2, activation_fn=lrelu, normalizer_fn=ly.batch_norm)
    else:
      img = ly.conv2d(img, num_outputs=2*Dim, kernel_size=3,stride=2, activation_fn=lrelu)
    if bn :
      img = ly.conv2d(img, num_outputs=4*Dim, kernel_size=3,stride=2, activation_fn=lrelu)
    else:
      img = ly.conv2d(img, num_outputs=4*Dim, kernel_size=3,stride=2, activation_fn=lrelu)
    if bn :
      img = ly.conv2d(img, num_outputs=8*Dim, kernel_size=3,stride=2, activation_fn=lrelu)
    else:
      img = ly.conv2d(img, num_outputs=8*Dim, kernel_size=3,stride=2, activation_fn=lrelu)
    
    logit = ly.fully_connected(tf.reshape(img, [batch_size, -1]), 1, activation_fn=None)
    return logit

In [ ]:
def critic_mlp(img, reuse=False):
  with tf.variable_scope('critic') as scope:
    if reuse:
      scope.reuse_variables()
    img = ly.fully_connected(tf.reshape(img, [batch_size, -1]), 4*Dim, activation_fn=tf.nn.relu)
    img = ly.fully_connected(img, 4*Dim, activation_fn=tf.nn.relu)
    img = ly.fully_connected(img, Dim, activation_fn=tf.nn.relu)
    logit = ly.fully_connected(img, 1, activation_fn=None)
    return logit

In [ ]:
def wgan_loss(logits_real, logits_fake, batch_size, real_data, fake_data, critic):
  
  print(real_data.shape)
  print(fake_data.shape)
  if mode == "wgan":
    c_loss = -tf.reduce_mean(logits_real) + tf.reduce_mean(logits_fake)
    g_loss = -tf.reduce_mean(logits_fake)

  elif mode == "wgan-gp":
    c_loss = -tf.reduce_mean(logits_real) + tf.reduce_mean(logits_fake)
    g_loss = -tf.reduce_mean(logits_fake)

    alpha = tf.random_uniform([batch_size,1,1,1], minval=0.0, maxval=1.0) 
    differences = fake_data - real_data
    interpolates = real_data + (alpha*differences)
    gradients = tf.gradients(critic(interpolates,reuse=True), [interpolates])[0]
    slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
    gradient_penalty = tf.reduce_mean((slopes-1.)**2)
    c_loss += lam*gradient_penalty
  elif mode == "dcgan":
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_fake, labels=tf.ones_like(logits_fake)))
    c_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_fake, labels=tf.zeros_like(logits_fake)))
    c_loss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_real, labels=tf.ones_like(logits_real)))
    c_loss/=2
  
  else:
    print('------')
  return c_loss, g_loss

In [ ]:
def model():
  
  x = tf.placeholder(dtype=tf.float32, shape=(batch_size, 32, 32, channel))
  z = sample_noise(batch_size, noise_dim)
  generator = generator_mlp if is_mlp else generator_conv
  critic = critic_mlp if is_mlp else critic_conv
  # critic = discriminator
  train = generator(batch_size,z)
  
  logits_real = critic(x)
  logits_fake = critic(train, reuse=True)

  c_loss, g_loss = wgan_loss(logits_real, logits_fake, batch_size, x, train, critic)

  g_loss_sum = tf.summary.scalar("g_loss", g_loss)
  c_loss_sum = tf.summary.scalar("c_loss", c_loss)
  img_sum = tf.summary.image("img", train, max_outputs=16)

  C_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'critic')
  G_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'generator')


  global_steps = tf.Variable(0, trainable=False)
  if mode == 'wgan-gp':
    opt_c = ly.optimize_loss(loss=c_loss, learning_rate=learning_rate_C,optimizer=partial(tf.train.AdamOptimizer, beta1=0.5, beta2=0.9), variables=C_vars, global_step=global_steps, summaries = ['gradient_norm'])
    opt_g = ly.optimize_loss(loss=g_loss, learning_rate=learning_rate_G,optimizer=partial(tf.train.AdamOptimizer, beta1=0.5, beta2=0.9), variables=G_vars, global_step=global_steps, summaries = ['gradient_norm'])
  elif mode == 'dcgan':
    opt_c = ly.optimize_loss(loss=c_loss, learning_rate=learning_rate_C,optimizer=partial(tf.train.AdamOptimizer, beta1=0.5), variables=C_vars, global_step=global_steps, summaries = ['gradient_norm'])
    opt_g = ly.optimize_loss(loss=g_loss, learning_rate=learning_rate_G,optimizer=partial(tf.train.AdamOptimizer, beta1=0.5), variables=G_vars, global_step=global_steps, summaries = ['gradient_norm'])
  elif mode == 'wgan':
    opt_c = ly.optimize_loss(loss=c_loss, learning_rate=learning_rate_C,optimizer=tf.train.RMSPropOptimizer, variables=C_vars, global_step=global_steps, summaries = ['gradient_norm'])
    opt_g = ly.optimize_loss(loss=g_loss, learning_rate=learning_rate_G,optimizer=tf.train.RMSPropOptimizer, variables=G_vars, global_step=global_steps, summaries = ['gradient_norm'])
    clipped_var_c = [tf.assign(var, tf.clip_by_value(var, clamp_lower, clamp_upper)) for var in C_vars]
    # merge the clip operations on critic variables
    with tf.control_dependencies([opt_c]):
      opt_c = tf.tuple(clipped_var_c)
  return opt_g, opt_c, x, train, c_loss, g_loss
  # d_extra_step = tf.get_collection(tf.GraphKeys.UPDATE_OPS, 'critic')
  # g_extra_step = tf.get_collection(tf.GraphKeys.UPDATE_OPS, 'generator')

In [ ]:
def next_feed_dict(real_data, dataset):
  train_img = dataset.train.next_batch(batch_size)[0]
  if data == 'cifar':
    train_img = train_img.reshape((batch_size, 3, 32, 32))
    train_img = train_img.transpose((0, 2, 3, 1))
  train_img = 2*train_img-1
  if data == 'mnist':
    train_img = np.reshape(train_img, (-1, 28, 28))
    npad = ((0, 0), (2, 2), (2, 2))
    train_img = np.pad(train_img, pad_width=npad,mode='constant', constant_values=-1)
    train_img = np.expand_dims(train_img, -1)
  
  feed_dict = {real_data: train_img}
  return feed_dict

In [ ]:
def main():
  
  with tf.device(device):
    opt_g, opt_c, real_data, fake_data, c_loss, g_loss =model()
  if FID:
    if data == 'cifar':
      stats_path = '/content/stat/fid_stats_cifar10_train.npz'
    elif data == 'svhn':
      stats_path = '/content/stat/fid_stats_svhn_train.npz'
    f = np.load(stats_path)
    mu_real, sigma_real = f['mu'][:], f['sigma'][:]
    f.close()
    inception_path = fid.check_or_download_inception(None) # download inception network
    fid.create_inception_graph(inception_path)  # load the graph into the current TF graph

  if data=='svhn':
    dataset = load_svhn()
  elif data=='mnist':
    dataset = input_data.read_data_sets('MNIST_data', one_hot=True)
  elif data=='cifar':
    dataset = load_cifar() 
  merged_all = tf.summary.merge_all()
  saver = tf.train.Saver()
  config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
  #config.gpu_options.allow_growth = True
  #config.gpu_options.per_process_gpu_memory_fraction = 0.8
  with tf.Session(config=config) as sess:
  #with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(log_dir, sess.graph)
    fid_tfvar = tf.Variable(0.0, trainable=False)
    fid_sum = tf.summary.scalar("FID", fid_tfvar)
    
    for i in range(max_iter_step):
      #if i < 25 or i % 500 == 0:
       # citers = 100
      start_time = time.time()
      if mode =='dcgan':
        citers = 1
      else:
        citers = Citers
      # Critic
      for j in range(citers):
        feed_dict = next_feed_dict(real_data, dataset) #
        if i%100 == 99 and j == 0:
          run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
          run_metadata = tf.RunMetadata()

          _, merged = sess.run([opt_c, merged_all], feed_dict=feed_dict,options=run_options, run_metadata=run_metadata)
          summary_writer.add_summary(merged, i)
          summary_writer.add_run_metadata(run_metadata, 'critic_metadata {}'.format(i), i)
        
        else:
          _=sess.run(opt_c, feed_dict=feed_dict)
      # Generator
      feed_dict = next_feed_dict(real_data, dataset)
      if i % 100 == 99:
        _, merged = sess.run([opt_g, merged_all], feed_dict=feed_dict,options=run_options, run_metadata=run_metadata)
        summary_writer.add_summary(merged, i)
        summary_writer.add_run_metadata(run_metadata, 'generator_metadata {}'.format(i), i)
        
      else:
        sess.run(opt_g, feed_dict=feed_dict) 
      if i % 1000 == 999:
        saver.save(sess, os.path.join(ckpt_dir, "model.ckpt"), global_step=i)
      

      if(i % 500 == 0):
       print("i:",i)
       _fake_data = sess.run(fake_data)
       _fake_data = (_fake_data+1.0)/2
       fig = disp_images(_fake_data,i)
       plt.show()
       #resize_function(fake_data, i)
      if i % 500 == 499 and IS:
        generator = generator_mlp if is_mlp else generator_conv
        tf.get_variable_scope().reuse_variables()
        samples_100 = generator(100)
        def get_inception_score():
          all_samples = []
          for i in xrange(10):
            all_samples.append(sess.run(samples_100))
          all_samples = np.concatenate(all_samples, axis=0)
          all_samples = ((all_samples+1.)*(255./2)).astype('int32')
          #all_samples = all_samples.reshape((-1, 3, 32, 32)).transpose(0,2,3,1)
          return fget_inception_score(list(all_samples))
        inception_score = get_inception_score()
        #print('inception score', inception_score[0])
        plot.plot('inception score', inception_score[0])

      if ((i < 5) or (i % 500 == 499)) and IS:
          plot.flush()
      if IS:
        plot.tick()

      if i % FID_STEP == 0:
        samples = np.zeros((FID_EVAL_SIZE, s,s,channel))
        n_fid_batches = FID_EVAL_SIZE // FID_SAMPLE_BATCH_SIZE
        #n_fid_batches = 5
        generator = generator_mlp if is_mlp else generator_conv
        tf.get_variable_scope().reuse_variables()
        for j in range(n_fid_batches):
          print("\rgenerate fid sample batch %d/%d " % (j + 1, n_fid_batches), end="", flush=True)
          frm = j * FID_SAMPLE_BATCH_SIZE
          to = frm + FID_SAMPLE_BATCH_SIZE

          samples[frm:to] = sess.run(generator(FID_SAMPLE_BATCH_SIZE))

        # Cast, reshape and transpose (BCHW -> BHWC)
        samples = ((samples + 1.0) * 127.5).astype('uint8')
        #samples = samples.reshape(FID_EVAL_SIZE, 3, DIM, DIM)
        #samples = samples.transpose(0,2,3,1)
        mu_gen, sigma_gen = fid.calculate_activation_statistics(samples,sess,batch_size=FID_BATCH_SIZE,verbose=True)
        
        print("calculate FID:", end=" ", flush=True)
        try:
          FIDv = fid.calculate_frechet_distance(mu_gen, sigma_gen, mu_real, sigma_real)
        except Exception as e:
          print(e)
          FIDv=500
        print(FIDv)
        sess.run(tf.assign(fid_tfvar, FIDv))
        summary_str = sess.run(fid_sum)
        summary_writer.add_summary(summary_str, i)






In [ ]:
main()

In [ ]:
#!cp '/content/log_wgan/eventsfidtest.out.tfevents.1643349973.36aea13ece8b' '/content/drive/MyDrive/test/result' 

In [ ]:
!cp -r "/content/log_wgan" "/content/drive/MyDrive/test/result/dcgan-svhn-fid"
#!cp -r "/content/ckpt_wgan" "/content/drive/MyDrive/test/result/wgan-gp-svhn-fid"
!cp -r "/content/dcgan_svhn" "/content/drive/MyDrive/test/result/dcgan-svhn-fid"
#cp "/content/log.pkl" "/content/drive/MyDrive/test/result/wgan-gp-svhn-fid"
#cp "/content/inception_score.jpg" "/content/drive/MyDrive/test/result/wgan-gp-svhn-fid"

In [ ]:
import pylab
read_file=open('log.pkl','rb')  
img=pickle.load(read_file)
read_file.close() 
for name in img:
  x_vals = np.sort(list(img[name].keys()))
  y_vals = [img[name][x] for x in x_vals]
  plt.clf()
  plt.plot(x_vals, y_vals)
  plt.xlabel('iteration')
  plt.ylabel(name)





In [ ]:
'''train_img = dataset1.train.next_batch(batch)[0]
img =train_img[1]
print(img.shape)
plt.imshow(img)
#img = img.reshape((3, 32, 32)) # 将一维向量改变形状得到这样一个元组:(高,宽,通道数)
#img = img.transpose((1, 2, 0))
#print(img.shape)
channel = 3
'''

In [ ]:
'''batch = 16
channel = 3

train_img = dataset.train.next_batch(batch)[0]

print(train_img.shape)
img = train_img[1] 
print(img.shape)
img = img.reshape((3, 32, 32)) # 将一维向量改变形状得到这样一个元组:(高,宽,通道数)
img = img.transpose((1, 2, 0)) 
print(img.shape)
plt.imshow(img)

channel = 3


train_img = train_img.reshape((batch, 3, 32, 32))
train_img = train_img.transpose((0, 2, 3, 1)) 
print(train_img[0])
print('train shape',train_img.shape)
disp_images(train_img)'''

In [ ]:
#!cp -r "/content/drive/MyDrive/test/cifar-10-batches-py" "/content/cifar10"